## Import library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

## Read data set

In [2]:
dtypes = {'id':'int64','item_nbr':'int32','store_nbr':'int8'}
path = 'file/'
#path = '../input/'
train = {'train':pd.read_csv(path+'train.csv',usecols=[1,2,3,4], dtype = dtypes, parse_dates = ['date'],
                    
                    skiprows = range(1,10168879))} # skip dates before 2017-01-01


In [3]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [ ]:
train.loc[train['unit_sales']<0,'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] = train['unit_sales'].apply(pd.np.log1p)

gc.collect()

In [ ]:
#fill gap in data
u_dates = train['date'].unique()
u_stores = train['store_nbr'].unique()
u_items = train['item_nbr'].unique()
train.set_index(['date','store_nbr','item_nbr'],inplace=True)
train = train.reindex(pd.MultiIndex.from_product([u_dates,u_stores,u_items],
                     names=['date','store_nbr','item_nbr']))

del u_dates, u_stores, u_items

train.loc[:,'unit_sales'].fillna(0, inplace = True)
train.reset_index(inplace = True)
last_date = train.iloc[train.shape[0]-1].date


##  load test

In [ ]:
test = pd.read_csv(path+'test.csv', dtype = dtypes)
test = test.set_index(['item_nbr','store_nbr'])
ltest = test.shape[0]

## Moving average

In [ ]:
for i in [1,3,7,14,28,56,112,224]:
    val = 'MA'+str(i)
    tmp = train[train['date']>lastdate - timedelta(int(i))]
    tmp1 = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame(val)
    test = test.join(tmp, how='left')

In [ ]:
# Median MA
test['unit_sales'] = test.iloc[:,1].median(axis=1)
test.loc[:,'unit_sales'].fillna(0,inplace = True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # retoring 
test[['id','unit_sales']].to_csv('median_ma.csv.gz',index=False, 
    float_format='%.3f',compression = 'gzip')


## Submission

In [ ]:
pd.read_csv(path + 'test.csv', usecols=[0,2,3,4], dtype = {'onpromotion': str}).set_index(
['item_nbr','store_nbr','onpromotion']).join(
train,how = 'left').fillna(0).to_csv('mean.csv',index=None)
